In [69]:
import pickle
import torch

In [70]:
folder="/data/home/umang/Trajectory_project/GPS-MTM/outputs/test_haystac/2025-08-28_18-13-53/test_outputs/random_masking_0.5_testing/ID"
pred_pickle_path="{}/predictions_batch_5000.pkl".format(folder)
batch_pickle_path = "{}/ground_truth_batch_5000.pkl".format(folder)
attention_pickle_path = "{}/attention_masks_batch_5000.pkl".format(folder)
masks_pickle_path = "{}/masks_batch_0.pkl".format(folder)
with open(pred_pickle_path, 'rb') as f:
    predictions = pickle.load(f)

with open(batch_pickle_path, 'rb') as f:
    batch = pickle.load(f)

with open(attention_pickle_path, 'rb') as f:
    attention_masks = pickle.load(f)

with open(masks_pickle_path, 'rb') as f:
    masks = pickle.load(f)

In [71]:
masks['states'].shape

torch.Size([221])

In [72]:
predictions["states"].shape

torch.Size([1, 221, 28])

In [73]:
predictions["actions"].shape

torch.Size([1, 221, 11])

In [74]:
# get the most frequently top-5 most repeated in ground_truth_list_States
# #don't use torch top-k..
ground_truth_list_states = []
for i in range(len(attention_masks)):
     # first zero mask is first time when attention_masks[i] becomes zero
    zero_indices = (attention_masks[i].flatten() == 0).nonzero(as_tuple=True)[0]
    first_zero_mask = zero_indices[0].item() if len(zero_indices) > 0 else attention_masks[i].numel()
    
    ground_truth_states= batch["states"][i, :first_zero_mask, :]
    ground_truth_states = torch.argmax(ground_truth_states, dim=-1)
    ground_truth_list_states.append(ground_truth_states)

k=4
ground_truth_list_states = torch.cat(ground_truth_list_states).flatten()
unique, counts = torch.unique(ground_truth_list_states, return_counts=True)
top_k_ground_truth = unique[torch.topk(counts, k=k).indices]
top_k_ground_truth

top_1_ground_truth = unique[torch.topk(counts, k=1).indices]
top_4_ground_truth = unique[torch.topk(counts, k=4).indices]    
# top_5_ground_truth = unique[torch.topk(counts, k=5).indices]
# top_10_ground_truth = unique[torch.topk(counts, k=10).indices]
# top_20_ground_truth = unique[torch.topk(counts, k=20).indices]


In [75]:
batch['actions'][0]

tensor([[ 6.4002e+04,  1.0000e+00,  0.0000e+00,  ...,  9.7963e-01,
         -1.8511e+00, -7.0780e-03],
        [ 6.4002e+04,  1.0000e+00,  0.0000e+00,  ...,  1.2759e+00,
         -1.2606e+00, -7.0780e-03],
        [ 6.4002e+04,  1.0000e+00,  0.0000e+00,  ...,  9.7963e-01,
         -1.8511e+00, -6.6465e-01],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])

In [76]:
masks

{'states': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0

In [77]:
predictions_list_states=[]
predictions_list_actions=[]
ground_truth_list_states=[]
ground_truth_list_actions=[]
correct_list=[]


# Get mask for this sequence
mask_states = masks["states"] == 1
mask_actions = masks["actions"] == 1

net_masks_states= attention_masks* masks["states"]
net_masks_actions= attention_masks* masks["actions"]

# Apply masks to get only positions where mask is 1
predictions_states = predictions["states"][net_masks_states.bool(), :]
predictions_actions = predictions["actions"][net_masks_actions.bool(), :]
ground_truth_states = batch["states"][net_masks_states.bool(), :]
ground_truth_actions = batch["actions"][net_masks_actions.bool(), :]

# take argmax of predictions states on last dimension

predictions_list_states.append(torch.argmax(predictions_states, dim=-1))
predictions_list_actions.append(torch.argmax(predictions_actions, dim=-1))
ground_truth_list_states.append(torch.argmax(ground_truth_states, dim=-1))
ground_truth_list_actions.append(torch.argmax(ground_truth_actions, dim=-1))

print(ground_truth_states)
correct = (predictions_states == ground_truth_states).float()
correct_list.append(correct)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [78]:
net_masks_states[..., 0:100]

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=torch.float64)

In [79]:
predictions_states[0:5, :]

tensor([[ -3.9841,  -7.1141,  -3.8031,  -0.6775,  -3.4464,  -7.6579,  -2.4293,
          -4.3889,  -7.2977,  -2.4098,  -2.2652,  -9.5652,  -3.5005,  -2.0540,
          -4.8445,  -9.4350,   0.4818,  -9.0604,  -1.9972,  -9.4097,  -2.5872,
         -13.3960,   3.5602,  -3.5836,  -3.4553,  -0.4803,  -2.5592,  -3.9427],
        [ -2.5780,  -7.0448,  -2.7969,  -1.1122,  -2.6556,  -8.0755,  -1.9673,
          -5.1917,  -6.9576,  -1.5807,  -1.5871,  -9.1667,  -4.3079,  -2.2315,
          -4.2874, -10.3568,   0.2844,  -8.1788,  -1.1772,  -9.2855,  -2.4063,
         -11.5044,   1.9791,  -2.1299,  -3.8009,  -0.2547,  -1.9919,  -2.8733],
        [ -1.0039,  -6.1492,  -2.2548,  -0.0539,  -1.4674,  -7.2733,  -1.0231,
          -3.2229,  -3.4613,  -1.0903,  -0.8755,  -6.5588,  -4.9496,  -2.0548,
          -3.1250,  -8.6848,   1.1061,  -6.4396,  -2.0252,  -7.1727,   0.0882,
         -10.2501,   3.3186,  -1.4977,  -3.2768,  -0.7507,  -1.7184,  -2.0916],
        [ -2.3711,  -4.3417,  -1.0993,   0.8341, 

In [80]:
predictions_states.shape,ground_truth_states.shape

(torch.Size([44, 28]), torch.Size([44, 28]))

In [81]:
predictions_list_states=torch.cat(predictions_list_states).flatten()
ground_truth_list_states=torch.cat(ground_truth_list_states).flatten()
predictions_list_states.shape, ground_truth_list_states.shape

(torch.Size([44]), torch.Size([44]))

In [82]:
predictions_list_states[0:200], ground_truth_list_states[0:200]

(tensor([22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
         22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 16,
         16, 16, 22, 22, 22, 22, 22, 16]),
 tensor([22, 10, 22,  6, 22, 10, 22,  0, 22, 10, 22, 10,  9, 22, 18, 22, 22, 22,
          4,  9, 16, 22, 22, 22, 10, 22, 10, 22, 22, 10,  9, 22,  0, 22, 10, 22,
         22, 16, 22, 16, 22, 10, 22, 10]))

In [83]:
print(net_masks_states[..., 0:300])

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]

In [84]:
predictions_list_states[0:200]-ground_truth_list_states[0:200]

tensor([ 0, 12,  0, 16,  0, 12,  0, 22,  0, 12,  0, 12, 13,  0,  4,  0,  0,  0,
        18, 13,  6,  0,  0,  0, 12,  0, 12,  0,  0, 12, 13,  0, 22,  0, 12, -6,
        -6,  0,  0,  6,  0, 12,  0,  6])

In [85]:
## Calculate accuracy USING predicted states and ground truth list states.
total_correct = 0
total_elements = 0
for pred_states, gt_states in zip(predictions_list_states, ground_truth_list_states):
    correct = (pred_states == gt_states).float()
    total_correct += correct.sum().item()
    total_elements += correct.numel()
accuracy = total_correct / total_elements if total_elements > 0 else 0
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.5000


In [86]:
# filter predictions and ground truth for each unique class
unique_classes = torch.unique(ground_truth_list_states)
filtered_predictions = {}
filtered_ground_truth = {}
for cls in unique_classes:
    mask = (ground_truth_list_states == cls)
    filtered_predictions[cls.item()] = predictions_list_states[mask]
    filtered_ground_truth[cls.item()] = ground_truth_list_states[mask]

# sort classes by frequency
class_counts = []
for cls in unique_classes:
    count = (ground_truth_list_states == cls).sum().item()
    class_counts.append((cls.item(), count))

# sort by count in descending order
class_counts.sort(key=lambda x: x[1], reverse=True)

# compute and print accuracy, precision, recall, and f1 score for each class sorted by frequency
print(f"{'Class':<5} {'Count':<6} {'Accuracy':<8} {'Precision':<9} {'Recall':<6} {'F1':<6}")
print("-" * 50)

for cls, count in class_counts:
    # Get predictions and ground truth for this class
    cls_predictions = filtered_predictions[cls]
    cls_ground_truth = filtered_ground_truth[cls]
    
    # Accuracy for this class
    accuracy = (cls_predictions == cls_ground_truth).float().mean().item()
    
    # For precision, recall, F1: need to consider this class vs all others
    # True positives: predicted this class and actually this class
    tp = ((predictions_list_states == cls) & (ground_truth_list_states == cls)).sum().item()
    
    # False positives: predicted this class but actually not this class
    fp = ((predictions_list_states == cls) & (ground_truth_list_states != cls)).sum().item()
    
    # False negatives: didn't predict this class but actually this class
    fn = ((predictions_list_states != cls) & (ground_truth_list_states == cls)).sum().item()
    
    # Calculate precision, recall, F1
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    print(f"{cls:<5} {count:<6} {accuracy:<8.4f} {precision:<9.4f} {recall:<6.4f} {f1:<6.4f}")

Class Count  Accuracy Precision Recall F1    
--------------------------------------------------
22    23     0.9130   0.5250    0.9130 0.6667
10    10     0.0000   0.0000    0.0000 0.0000
9     3      0.0000   0.0000    0.0000 0.0000
16    3      0.3333   0.2500    0.3333 0.2857
0     2      0.0000   0.0000    0.0000 0.0000
4     1      0.0000   0.0000    0.0000 0.0000
6     1      0.0000   0.0000    0.0000 0.0000
18    1      0.0000   0.0000    0.0000 0.0000


In [87]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# Convert to numpy for sklearn compatibility
gt_numpy = ground_truth_list_states.cpu().numpy()
pred_numpy = predictions_list_states.cpu().numpy()

# Overall metrics
overall_accuracy = accuracy_score(gt_numpy, pred_numpy)
macro_f1 = f1_score(gt_numpy, pred_numpy, average='macro')
weighted_f1 = f1_score(gt_numpy, pred_numpy, average='weighted')
macro_precision = precision_score(gt_numpy, pred_numpy, average='macro')
weighted_precision = precision_score(gt_numpy, pred_numpy, average='weighted')
macro_recall = recall_score(gt_numpy, pred_numpy, average='macro')
weighted_recall = recall_score(gt_numpy, pred_numpy, average='weighted')

print("Overall Performance Metrics:")
print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Macro F1 Score: {macro_f1:.4f}")
print(f"Weighted F1 Score: {weighted_f1:.4f}")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Weighted Precision: {weighted_precision:.4f}")
print(f"Macro Recall: {macro_recall:.4f}")
print(f"Weighted Recall: {weighted_recall:.4f}")

print("\nDetailed Classification Report:")
print(classification_report(gt_numpy, pred_numpy))

# Class-wise metrics accounting for imbalance
unique_classes = torch.unique(ground_truth_list_states)
class_counts = [(cls.item(), (ground_truth_list_states == cls).sum().item()) for cls in unique_classes]
class_counts.sort(key=lambda x: x[1], reverse=True)

print(f"\n{'Class':<5} {'Count':<6} {'Freq%':<6} {'Precision':<9} {'Recall':<6} {'F1':<6}")
print("-" * 50)

for cls, count in class_counts:
    freq_pct = (count / len(ground_truth_list_states)) * 100
    
    # Calculate metrics for this specific class
    cls_mask_gt = (gt_numpy == cls)
    cls_mask_pred = (pred_numpy == cls)
    
    tp = ((pred_numpy == cls) & (gt_numpy == cls)).sum()
    fp = ((pred_numpy == cls) & (gt_numpy != cls)).sum()
    fn = ((pred_numpy != cls) & (gt_numpy == cls)).sum()
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    print(f"{cls:<5} {count:<6} {freq_pct:<6.1f} {precision:<9.4f} {recall:<6.4f} {f1:<6.4f}")

Overall Performance Metrics:
Overall Accuracy: 0.5000
Macro F1 Score: 0.1190
Weighted F1 Score: 0.3680
Macro Precision: 0.0969
Weighted Precision: 0.2915
Macro Recall: 0.1558
Weighted Recall: 0.5000

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         3
          10       0.00      0.00      0.00        10
          16       0.25      0.33      0.29         3
          18       0.00      0.00      0.00         1
          22       0.53      0.91      0.67        23

    accuracy                           0.50        44
   macro avg       0.10      0.16      0.12        44
weighted avg       0.29      0.50      0.37        44


Class Count  Freq%  Precision Recall F1    
--------------------------------------------------
22    23     52.3   0

/data/home/umang/miniconda3/envs/mtm/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/data/home/umang/miniconda3/envs/mtm/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/data/home/umang/miniconda3/envs/mtm/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{me

In [88]:
# get the number of correct predictions for classes in top_10_ground_truth
#top_k=[top_1_ground_truth, top_5_ground_truth, top_10_ground_truth, top_20_ground_truth]
top_k=[top_1_ground_truth, top_4_ground_truth]
for i in range(len(top_k)):
    filtered_gt = ground_truth_list_states[(ground_truth_list_states.unsqueeze(-1) == top_k[i]).any(dim=-1)]
    filtered_pred = predictions_list_states[(ground_truth_list_states.unsqueeze(-1) == top_k[i]).any(dim=-1)]

    # get the number of correct predictions and accuracy
    correct_predictions = (filtered_pred == filtered_gt).sum()
    accuracy = correct_predictions / filtered_gt.numel()
    print(f"Number of correct predictions in {top_k[i]} ground truth: {correct_predictions.item()}")
    print(f"Accuracy of correct predictions in {top_k[i]} ground truth: {accuracy.item()}")

Number of correct predictions in tensor([22]) ground truth: 21
Accuracy of correct predictions in tensor([22]) ground truth: 0.9130434989929199
Number of correct predictions in tensor([22, 10, 16,  9]) ground truth: 22
Accuracy of correct predictions in tensor([22, 10, 16,  9]) ground truth: 0.5641025900840759


### Proof that model is working properly below.

-0.0207 is the index for mask value to be filled in. Corresponding mask is shown below.

In [89]:
# (Pdb) x_sample[0,:,0]
# tensor([-0.9309, -2.4709, -0.9647, -2.5226, -0.9906, -2.6224, -1.0323, -2.6396,
#         -1.0287, -2.5965, -0.0207, -0.0207, -1.0252, -2.4983, -1.0098, -0.0207,
#         -1.0417, -2.3514, -0.0207, -0.0207, -0.0207, -2.3262, -0.9608, -2.4531,
#         -0.9609, -2.5430, -0.0207, -2.4288, -0.0207, -1.5484, -0.9251, -2.4177,
#         -0.0207, -0.0207, -0.8637, -2.3879, -0.8872, -0.0207, -0.9029, -2.3117,
#         -0.9281, -2.3534, -1.0126, -2.4598, -0.9165, -0.0207, -0.8610, -2.4906,
#         -0.0207, -2.5534, -1.0234, -0.8551, -2.4868, -0.8567, -2.4971, -0.9002,
#         -2.4162, -0.0207, -0.0207, -0.9232, -0.0207, -0.9814, -0.0207, -0.0207,
#         -2.3403, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207, -0.0207,
#         -0.0207, -0.0207, -0.0207, -0.0207, -0.0207], device='cuda:0')

In [90]:
# (Pdb) masks['states']
# tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
#         1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
#         0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
#         1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
#         1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
#         0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
#         1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
#         1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#         1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
#         1, 1, 0, 1, 1], device='cuda:0')